In [1]:
{-# LANGUAGE BinaryLiterals #-}
{-# LANGUAGE NumericUnderscores #-}

import qualified Data.Vector.Unboxed as V
import Data.Word
import qualified Data.ByteString as BS
import Data.Bits

In [2]:
data VM = VM
    { vmMemory :: V.Vector Word16
    , vmR1
    , vmR2
    , vmR3
    , vmR4
    , vmPC
    , vmSP :: Word16
    } deriving (Eq, Show)

In [3]:
data Opcode
    = NULL
    | ADD
    | SUB
    | MUL
    | DIV
    | LOAD
    | STORE
    | LOADi
    | STOREi
    deriving (Eq, Show)

data Register = R1 | R2 | R3 | R4 deriving (Eq, Show, Ord, Enum)

data Instruction = Instruction
    { instructionOpcode :: Opcode
    , instructionR1
    , instructionR2 :: Register
    , instructionData :: Word8
    } deriving (Eq, Show)

In [4]:
oPMASK, r1MASK, r2MASK, dTMASK :: Word16
oPMASK = 0b11110000_00000000
r1MASK = 0b00001100_00000000
r2MASK = 0b00000011_00000000
dTMASK = 0b00000000_11111111

In [5]:
parseOpcode :: Word8 -> Opcode
parseOpcode w = case w of
    0 -> NULL
    3 -> ADD
    4 -> SUB
    5 -> MUL
    6 -> DIV
    7 -> LOAD
    8 -> STORE
    9 -> LOADi
    10 -> STOREi
    _ -> error "invalid opcode"

parseRegister :: Word8 -> Register
parseRegister w = case w of
    0 -> R1
    1 -> R2
    2 -> R3
    3 -> R4
    _ -> error "invalid register"
    
parseInstruction :: Word16 -> Instruction
parseInstruction w = let
    instrData =               fromIntegral $                 w .&. dTMASK
    instrR2 = parseRegister . fromIntegral $ shiftR         (w .&. r2MASK) 8
    instrR1 = parseRegister . fromIntegral $ shiftR (shiftR (w .&. r1MASK) 8) 2
    instrOp = parseOpcode   . fromIntegral $ shiftR (shiftR (w .&. oPMASK) 8) 4
    in Instruction instrOp instrR1 instrR2 (fromIntegral instrData)

In [6]:
addressRegister :: Register -> VM -> Word16
addressRegister r = case r of
    R1 -> vmR1
    R2 -> vmR2
    R3 -> vmR3
    R4 -> vmR4

storeRegister :: Word16 -> Register -> VM -> VM
storeRegister value r vm = case r of
    R1 -> vm { vmR1 = value }
    R2 -> vm { vmR2 = value }
    R3 -> vm { vmR3 = value }
    R4 -> vm { vmR4 = value }

step :: VM -> VM
step vm@VM{ vmMemory = memory, vmPC = pc } = let
    (Instruction op r1 r2 instrData) = parseInstruction (memory V.! fromIntegral pc)
    in case op of
        NULL -> vm
        ADD -> let
            result = addressRegister r1 vm + addressRegister r2 vm
            vm' = storeRegister result r1 vm
            in vm' { vmPC = pc + 1 }
        SUB -> let
            result = addressRegister r1 vm - addressRegister r2 vm
            vm' = storeRegister result r1 vm
            in vm' { vmPC = pc + 1 }
        MUL -> let
            result = addressRegister r1 vm * addressRegister r2 vm
            vm' = storeRegister result r1 vm
            in vm' { vmPC = pc + 1 }
        DIV -> let
            result = addressRegister r1 vm `div` addressRegister r2 vm
            vm' = storeRegister result r1 vm
            in vm' { vmPC = pc + 1 }
        LOAD -> let
            index = fromIntegral $ addressRegister r2 vm
            value = memory V.! index
            vm' = storeRegister value r1 vm
            in vm' { vmPC = pc + 1 }
        STORE -> let
            value = addressRegister r2 vm
            index = fromIntegral $ addressRegister r1 vm
            memory' = memory V.// [(index,value)]
            in vm { vmMemory = memory', vmPC = pc + 1 }
        LOADi -> let
            vm' = storeRegister (fromIntegral instrData) r1 vm
            in vm' { vmPC = pc + 1 }
        STOREi -> let
            index = fromIntegral $ addressRegister r1 vm
            memory' = memory V.// [(index, fromIntegral instrData)]
            in vm { vmMemory = memory', vmPC = pc + 1 }

In [7]:
loop :: VM -> VM
loop vm = let
    vm' = step vm
    in if vmPC vm' == vmPC vm
        then vm'
        else loop vm'

In [8]:
raw <- BS.readFile "./simplevm.rom"

In [9]:
unpacked = BS.unpack raw

In [10]:
formatLittleEndian' :: [Word16] -> [Word8] -> [Word16]
formatLittleEndian' acc [] = acc
formatLittleEndian' acc (b2:b1:bs) = let
    w16 = (fromIntegral b1 :: Word16) `shiftL` 8 .|. fromIntegral b2
    in formatLittleEndian' (acc ++ [w16]) bs
formatLittleEndian' _ _ = error "odd number of bytes"

In [11]:
formatLittleEndian = formatLittleEndian' []

In [12]:
formatted = formatLittleEndian unpacked


new program = let
    emptyVector = V.replicate 256 0 :: V.Vector Word16
    zipped = zip [0..] program
    loaded = emptyVector V.// zipped
    in VM { vmMemory = loaded, vmR1 = 0, vmR2 = 0, vmR3 = 0, vmR4 = 0, vmPC = 0, vmSP = 0  }

In [13]:
memory = vmMemory $ loop $ new formatted

In [14]:
assertions = 
    [ memory V.! 255 == 3
    , memory V.! 254 == 9
    , memory V.! 253 == 6
    , memory V.! 240 == 0xde
    , memory V.! 241 == 0xad
    , memory V.! 242 == 0xbe
    , memory V.! 243 == 0xef
    ]

and assertions

True